# Problem Statement

Given imaging studies, we want to be able to get the differences between them for Inteleshare testing purposes. For example, when deleting a study from a routed namespace, does it affect the original study in the main namespace?

# Solution
Given two paths to two imaging studies, the notebook compare them.

# Code
### 1. Setup Imaging studies path

In [ ]:
import difflib
import pydicom as pdc
from pathlib import Path

In [ ]:
first_study = Path(f"{Path.cwd()}/TESTSUBJT")
second_study = Path(f"{Path.cwd()}/TESTSUBJT (3)")

# 2. Match DICOM files to each other (from first to second study)

In [ ]:
# Iterate through first study directory, get all dicoms
# then get dicoms from second study based on first study's path

dicom_matches = []
img_dirs = [x for x in first_study.iterdir() if (x.is_dir()) and (x.name != "reports")]

for img_dir in img_dirs:
    dicoms = [x for x in img_dir.iterdir() if x.is_file()]

    for dicom in dicoms:
        matched_dicom = dicom.parents[2] / second_study.name / img_dir.name / dicom.name

        # Check if dicom exists in second img study
        if matched_dicom.exists():
            dicom_matches.append(
                {"first_study_dicom": dicom, "second_study_dicom": matched_dicom}
            )
        else:
            raise Exception(f"Matched dicom for {dicom} not found!")

dicom_matches

# 3. Compare the matches together

Example output:
```
  Dataset.file_meta -------------------------------

- (0002,0000) File Meta Information Group Length  UL: 190

+ (0002,0000) File Meta Information Group Length  UL: 192

  (0002,0001) File Meta Information Version       OB: b'\x00\x01'

- (0002,0002) Media Storage SOP Class UID         UI: MR Image Storage

+ (0002,0002) Media Storage SOP Class UID         UI: CT Image Storage

  (0002,0010) Transfer Syntax UID                 UI: Explicit VR Little Endian

  (0002,0012) Implementation Class UID            UI: 1.3.6.1.4.1.5962.2

  (0002,0013) Implementation Version Name         SH: 'DCTOOL100'

  (0002,0016) Source Application Entity Title     AE: 'CLUNIE1'

  -------------------------------------------------

- (0008,0018) SOP Instance UID                    UI: 1.3.6.1.4.1.5962.1.1.4.1.1.20040826185059.5457

+ (0008,0018) SOP Instance UID                    UI: 1.3.6.1.4.1.5962.1.1.1.1.1.20040119072730.12322

- (0008,0020) Study Date                          DA: '20040826'

+ (0008,0020) Study Date                          DA: '20040119'

- (0008,0021) Series Date                         DA: ''

+ (0008,0021) Series Date                         DA: '19970430'
```

In [ ]:
for dicom_match in dicom_matches:
    first_dicom = pdc.dcmread(dicom_match["first_study_dicom"])
    second_dicom = pdc.dcmread(dicom_match["second_study_dicom"])

    rep = []

    for ds in [first_dicom, second_dicom]:
        lines = str(ds).split("\n")
        lines = [line + "\n" for line in lines]
        rep.append(lines)

    diff = difflib.Differ()
    for line in diff.compare(rep[0], rep[1]):
        if line[0] != "?":
            print(line)